In [1]:
import numpy as np 
import pandas as pd
import xgboost as xgb 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LassoCV, Lasso, RidgeCV, ElasticNetCV, ElasticNet, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [2]:
# col from 1 to 10
def train_test_split(testID, data, col):
    col -= 1
    tid = testID.iloc[:, col].values.copy()
    tid -= 1
    train = data.drop(tid)
    test = data.iloc[tid, :].copy()
    test_y = test['Sale_Price']
    test.drop(['Sale_Price'], axis=1, inplace=True)
    return train, test, test_y

In [5]:
data = pd.read_csv('./Ames_data.csv')
testID = pd.read_csv('./project_testIDs.dat', header=None, sep=' ')

In [34]:
def model(train, test, test_y):
    # processing train data
    removed_features = ['Street', 'Utilities', 'Condition_2', 'Roof_Matl', 'Heating', 
                    'Pool_QC', 'Misc_Feature', 'Low_Qual_Fin_SF', 'Pool_Area', 
                    'Longitude','Latitude', 'Bsmt_Full_Bath', 'Bsmt_Half_Bath', 'Three_season_porch', 'Misc_Val']
    train.index = pd.Index(range(len(train)))
    train.fillna(0, inplace=True)
    Y = train['Sale_Price']
    train.drop(['Sale_Price', 'PID'], axis=1, inplace=True)
    train.drop(removed_features, axis=1, inplace=True)
    # feature
    features = train.columns.values.tolist()
    categ_features = []
    dense_features = []
    for f, dtype in train.dtypes.items():
        if dtype == object:
            categ_features.append(f)
        else:
            dense_features.append(f)
    train_reg = train.copy()
    # wins
    wins_features = ["Lot_Frontage", "Lot_Area", "Mas_Vnr_Area", "BsmtFin_SF_2", "Bsmt_Unf_SF", "Total_Bsmt_SF", 
                     "Second_Flr_SF", 'First_Flr_SF', "Gr_Liv_Area", "Garage_Area", "Wood_Deck_SF", 
                     "Open_Porch_SF", 'Enclosed_Porch', 'Screen_Porch'] 
    features_quan = []
    for f in wins_features:
        col = train_reg[f]
        quan95 = np.quantile(col, 0.95)
        features_quan.append(quan95)
        train_reg[f] = np.where(col > quan95, quan95, col)
    # standardize
    scaler = StandardScaler()
    train_reg_dense = train_reg[dense_features]
    standard_train_reg = pd.DataFrame(scaler.fit_transform(train_reg_dense), columns=train_reg_dense.columns)
    train_reg = standard_train_reg
    Y = np.log(Y)
    # processing test
    test.index = pd.Index(range(len(test)))
    test.fillna(0, inplace=True)
    test.drop(removed_features, axis=1, inplace=True)
    test_PID = test['PID']
    test.drop(['PID'], axis=1, inplace=True)
 
    test_reg = test.copy()
    # wins
    for i in range(len(wins_features)):
        quan95 = features_quan[i]
        f = wins_features[i]
        test_reg[f] = np.where(test_reg[f] > quan95, quan95, test_reg[f])
    # standardize
    test_reg_dense = test_reg[dense_features]
    standard_test_reg = pd.DataFrame(scaler.transform(test_reg_dense), columns=test_reg_dense.columns)
    test_reg = standard_test_reg
    
   #  return train_reg
    ridge_alphas = np.linspace(20, 30, 20) 
    ridgecv = RidgeCV(alphas=ridge_alphas, normalize=False, cv=10)
    ridgecv.fit(train_reg, Y)
    
    test_pred_reg = ridgecv.predict(test_reg)
    err_reg = np.sqrt(mean_squared_error(test_pred_reg, np.log(test_y)))
    # err_reg = np.sqrt(mean_squared_error(test_pred_reg, test_y))
    
    OLS = LinearRegression()
    OLS.fit(train_reg, Y)
    test_pred_ols = OLS.predict(test_reg)
    err_ols = np.sqrt(mean_squared_error(test_pred_ols, np.log(test_y)))
#     return np.round(err_ols, 4), OLS.intercept_
    
#     xgb_model = xgb.XGBRegressor(n_estimators=500, max_depth=5, learning_rate=0.05,
#                       verbosity=0, subsample=0.5, random_state=2021)

#     xgb_model.fit(train_reg, Y)
#     test_pred_xgb = xgb_model.predict(test_reg)
    
#     return np.round(np.sqrt(mean_squared_error(test_pred_xgb, np.log(test_y))), 4)
    lassocv = LassoCV(normalize=False, cv=10)
    lassocv.fit(train_reg, Y)
    test_pred_las = lassocv.predict(test_reg)
    err_las = np.sqrt(mean_squared_error(test_pred_las, np.log(test_y)))
    # err_las = np.sqrt(mean_squared_error(test_pred_las, test_y))
    

    encv = ElasticNetCV(normalize=False, cv=10)
    # encv = ElasticNet()
    encv.fit(train_reg, Y)
    test_pred_en = encv.predict(test_reg)
    err_en = np.sqrt(mean_squared_error(test_pred_en, np.log(test_y)))
    # err_en = np.sqrt(mean_squared_error(test_pred_en, test_y))
    return OLS, ridgecv, lassocv, encv, dense_features
    
    c1 = np.sum(ridgecv.coef_==0)
    c2 = np.sum(lassocv.coef_==0)
    c3 = np.sum(encv.coef_==0)
    return np.round(err_reg, 4), np.round(err_las, 4), np.round(err_en, 4), c1, c2, c3 

In [191]:
for i in range(1, 11):
    train, test, test_y = train_test_split(testID, data, i)
    x, y, z, a, b, c = model(train, test, test_y)
    e1.append(x)
    e2.append(y)
    e3.append(z)
    z1.append(a)
    z2.append(b)
    z3.append(c)

In [195]:
print(e1, np.mean(e1))
print(e2, np.mean(e2))
print(e3, np.mean(e3))
print(z1)
print(z2)
print(z3)

[0.1631, 0.162, 0.1545, 0.1608, 0.1447, 0.1749, 0.174, 0.1665, 0.1722, 0.1577] 0.16304
[0.1628, 0.1621, 0.1547, 0.1606, 0.1451, 0.1746, 0.174, 0.1666, 0.1719, 0.1579] 0.16302999999999998
[0.1629, 0.1621, 0.1547, 0.1606, 0.145, 0.1745, 0.174, 0.1666, 0.1719, 0.1579] 0.16302
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[3, 2, 2, 2, 2, 1, 3, 1, 3, 1]
[3, 2, 2, 2, 2, 2, 2, 1, 3, 2]


In [37]:
o, r, l, e, F = model(train, test, test_y)

In [40]:
df = pd.DataFrame({'fea': F, 'OLS':o.coef_, 'R':r.coef_, 'L':l.coef_, 'EN':e.coef_})

In [42]:
df.to_csv('coef.csv', index=False)